# **I. Model Selection & Baseline Leaderboard**

## **I.1. Objective & Experiment Design**

The objective of this phase is to establish a **performance baseline** using default algorithms. This "Leaderboard" serves two critical functions:

1.  **Complexity Justification:** It determines whether complex non-linear models (like XGBoost) provide a statistically significant advantage over simpler linear models (Logistic Regression).
2.  **Overfitting Detection:** By simultaneously measuring performance on the **Training Set** and the **Validation Set**, we identify models that "memorize" the data rather than generalizing the underlying cluster logic.

**The Candidate Algorithms:**

*   **Logistic Regression:** A linear baseline. If this performs well, the cluster boundaries are simple and linear.
*   **Random Forest:** A bagging ensemble. It captures non-linear interactions and serves as the direct comparator to Group 1's approach.
*   **XGBoost / LightGBM:** Gradient boosting machines. These are the current industry standard for tabular data, expected to handle the "Augmented Features" (like delivery delays) most effectively.

## **I.2. Environment Setup & Metric Definition**
To ensure rigorous evaluation, we define a standardized **Evaluation Function** that will be applied identically to all models.

**Key Metrics Selection:**
*   **F1-Score (Weighted):** The primary success metric. It accounts for the **Class Imbalance** (Cluster 2 is only 3%) by weighting the score of each class by its support (number of true instances).
*   **Log Loss (Cross-Entropy):** Measures the **confidence** of predictions. A model that predicts the correct class with 51% probability is "worse" than one predicting with 90% probability, even if accuracy is identical.
*   **ROC-AUC (One-vs-Rest):** Measures the model's ability to distinguish between classes across different probability thresholds.


**Code Implementation: Setup & Metrics**

**Purpose:** Import necessary libraries, load the stratified datasets created in ***Data Engineering*** section, and define a reusable function to calculate and display the required metrics.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn Models & Metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, log_loss, roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer

# Gradient Boosting Libraries
import xgboost as xgb
import lightgbm as lgb

In [2]:
# --- CONFIGURATION ---
DATA_DIR = "data/"

def load_stratified_data():
    """
    Loads the 6 split files generated in Task 3.
    Returns X_train, y_train, X_val, y_val, X_test, y_test
    """
    print("--- [START] Loading Stratified Data ---")
    try:
        X_train = pd.read_csv(f"{DATA_DIR}X_train.csv")
        y_train = pd.read_csv(f"{DATA_DIR}y_train.csv").squeeze() # squeeze to convert DF to Series
        X_val = pd.read_csv(f"{DATA_DIR}X_val.csv")
        y_val = pd.read_csv(f"{DATA_DIR}y_val.csv").squeeze()
        X_test = pd.read_csv(f"{DATA_DIR}X_test.csv")
        y_test = pd.read_csv(f"{DATA_DIR}y_test.csv").squeeze()
        
        print(f"    Train Shape: {X_train.shape}")
        print(f"    Val Shape:   {X_val.shape}")
        print(f"    Test Shape:  {X_test.shape}")
        print("--- [END] Data Loaded Successfully ---\n")
        return X_train, y_train, X_val, y_val, X_test, y_test
    except FileNotFoundError:
        print("    [ERROR] Split files not found. Please run Task 3 first.")
        return None, None, None, None, None, None

def evaluate_model(model, X, y, dataset_name="Validation"):
    """
    Calculates F1-Weighted, Log Loss, and ROC-AUC for a given model and dataset.
    Returns a dictionary of metrics.
    """
    # 1. Generate Predictions
    y_pred = model.predict(X)
    y_prob = model.predict_proba(X)
    
    # 2. Calculate Metrics
    # F1-Weighted handles class imbalance
    f1 = f1_score(y, y_pred, average='weighted')
    
    # Log Loss requires probability estimates
    try:
        ll = log_loss(y, y_prob)
    except ValueError:
        # Fallback if classes are missing in a small split (rare with stratification)
        ll = np.nan
        
    # ROC-AUC (One-vs-Rest) for Multiclass
    try:
        auc = roc_auc_score(y, y_prob, multi_class='ovr', average='weighted')
    except ValueError:
        auc = np.nan

    # 3. Print Results
    print(f"[{dataset_name} Performance]")
    print(f"    F1-Score (Weighted): {f1:.4f}")
    print(f"    ROC-AUC (OvR):       {auc:.4f}")
    print(f"    Log Loss:            {ll:.4f}")
    
    return {
        "F1_Weighted": f1,
        "ROC_AUC": auc,
        "Log_Loss": ll
    }

# Execute Loading
X_train, y_train, X_val, y_val, X_test, y_test = load_stratified_data()

--- [START] Loading Stratified Data ---
    Train Shape: (65350, 10)
    Val Shape:   (14004, 10)
    Test Shape:  (14004, 10)
--- [END] Data Loaded Successfully ---



## **I.3. Baseline Models: Logistic Regression & Random Forest**

*   **Logistic Regression:**
    *   **Role:** The "Simplicity Test."
    *   **Hypothesis:** If the clusters created by K-Means are geometrically distinct (e.g., separated by clear hyperplanes), this simple linear model should perform surprisingly well. If it fails (low F1 score), it confirms the relationships between inputs (like `credit_card_usage`) and clusters are **non-linear**.
    *   **Configuration:** We use `max_iter=1000` to ensure the solver converges, as our dataset has 93k rows.

*   **Random Forest Classifier:**
    *   **Role:** The "Direct Comparator."
    *   **Hypothesis:** This mimics Group 1's approach but applied to our **stratified** data. Random Forest handles non-linearity well but is prone to **overfitting** (high Train score, low Val score) if trees are allowed to grow too deep.
    *   **Configuration:** We use a `random_state` seed for reproducibility.

**Code Implementation: Training LR and RF**

**Purpose:** Train both models on `X_train`, then evaluate them on **both** `X_train` (to check for memorization) and `X_val` (to check for generalization).

In [3]:
# Store results for the Leaderboard comparison later
leaderboard = []

# --- 1. LOGISTIC REGRESSION (Linear Baseline) ---
print("\n=== Model 1: Logistic Regression ===")
# Initialize
lr_model = LogisticRegression(
    multi_class='multinomial', 
    solver='lbfgs', 
    max_iter=3000, 
    random_state=42
)

# Train
lr_model.fit(X_train, y_train)

# Evaluate on Train (Check for Underfitting)
print("  > Evaluating on TRAINING set...")
train_metrics_lr = evaluate_model(lr_model, X_train, y_train, dataset_name="Train")

# Evaluate on Validation (Check for Generalization)
print("  > Evaluating on VALIDATION set...")
val_metrics_lr = evaluate_model(lr_model, X_val, y_val, dataset_name="Validation")

# Record Results
leaderboard.append({
    "Model": "Logistic Regression",
    "Train_F1": train_metrics_lr["F1_Weighted"],
    "Val_F1": val_metrics_lr["F1_Weighted"],
    "Val_LogLoss": val_metrics_lr["Log_Loss"]
})


# --- 2. RANDOM FOREST (Bagging Ensemble) ---
print("\n=== Model 2: Random Forest (Default) ===")
# Initialize
rf_model = RandomForestClassifier(
    n_estimators=100, 
    random_state=42, 
    n_jobs=-1  # Use all CPU cores
)

# Train
rf_model.fit(X_train, y_train)

# Evaluate on Train
print("  > Evaluating on TRAINING set...")
train_metrics_rf = evaluate_model(rf_model, X_train, y_train, dataset_name="Train")

# Evaluate on Validation
print("  > Evaluating on VALIDATION set...")
val_metrics_rf = evaluate_model(rf_model, X_val, y_val, dataset_name="Validation")

# Record Results
leaderboard.append({
    "Model": "Random Forest",
    "Train_F1": train_metrics_rf["F1_Weighted"],
    "Val_F1": val_metrics_rf["F1_Weighted"],
    "Val_LogLoss": val_metrics_rf["Log_Loss"]
})


=== Model 1: Logistic Regression ===


d:\_Python\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
d:\_Python\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 3000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=3000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score (Weighted): 0.9505
    ROC-AUC (OvR):       0.9942
    Log Loss:            0.1144
  > Evaluating on VALIDATION set...
[Validation Performance]
    F1-Score (Weighted): 0.9506
    ROC-AUC (OvR):       0.9939
    Log Loss:            0.1184

=== Model 2: Random Forest (Default) ===
  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score (Weighted): 1.0000
    ROC-AUC (OvR):       1.0000
    Log Loss:            0.0075
  > Evaluating on VALIDATION set...
[Validation Performance]
    F1-Score (Weighted): 0.9951
    ROC-AUC (OvR):       0.9999
    Log Loss:            0.0229


**Analysis of Output**

1.  **The Logistic Regression Warning:**
    *   `lbfgs failed to converge`: This is expected. Even with 3000 iterations, the complex high-dimensional surface (10 features, 4 classes) makes it hard for a linear solver to find the absolute global minimum.
    *   **Performance:** Despite non-convergence, it hit **95% F1-Score**. This proves that the clusters have **very strong linear separation**. This is a key finding: *K-Means created mathematically distinct groups, so even a simple line can separate them well.*

2.  **The Random Forest "Perfect Score":**
    *   **Train F1 = 1.0000**: The model has perfectly memorized the training data. This is textbook **overfitting**.
    *   **Validation F1 = 0.9951**: However, the validation score is also near perfect.
    *   **Implication:** This confirms the "Surrogate Model" nature of the task. Because $Y$ (Cluster) was created from $X$ (Features), the mapping is deterministic. The Random Forest has successfully reverse-engineered the K-Means logic.

**Modification:** We do not need to "fix" the Logistic Regression convergence because it is just a baseline comparison. The fact that it struggles while RF succeeds validates moving to tree-based models.

## **I.4. Gradient Boosting Challengers: XGBoost & LightGBM**

To complete the baseline assessment, we deploy two industry-standard Gradient Boosting machines. Unlike Random Forest (which builds trees in parallel), these models build trees **sequentially**, with each new tree correcting the errors of the previous one.

*   **XGBoost (eXtreme Gradient Boosting):**
    *   **Strength:** Known for precision and regularization (preventing overfitting).
    *   **Hypothesis:** It should provide the best **Log Loss** (probability calibration) of all models. XGBoost should match Random Forest's accuracy (99%+) but potentially with lower **Log Loss** (better probability confidence).
*   **LightGBM (Light Gradient Boosting Machine):**
    *   **Strength:** Optimized for speed and efficiency using histogram-based learning.
    *   **Hypothesis:** It serves as a "Speed Test." If it matches XGBoost's accuracy but trains 5x faster, it becomes the preferred candidate for deployment.


In [ ]:
# --- 3. XGBOOST (Gradient Boosting) ---
print("\n=== Model 3: XGBoost Classifier ===")
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=4,
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)

print("  > Evaluating on TRAINING set...")
train_metrics_xgb = evaluate_model(xgb_model, X_train, y_train, dataset_name="Train")

print("  > Evaluating on VALIDATION set...")
val_metrics_xgb = evaluate_model(xgb_model, X_val, y_val, dataset_name="Validation")

leaderboard.append({
    "Model": "XGBoost",
    "Train_F1": train_metrics_xgb["F1_Weighted"],
    "Val_F1": val_metrics_xgb["F1_Weighted"],
    "Val_LogLoss": val_metrics_xgb["Log_Loss"]
})


# --- 4. LIGHTGBM (High-Efficiency Boosting) ---
print("\n=== Model 4: LightGBM Classifier ===")
# Force verbosity=-1 to suppress internal warnings
lgb_model = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=4,
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    verbosity=-1
)

lgb_model.fit(X_train, y_train)

print("  > Evaluating on TRAINING set...")
train_metrics_lgb = evaluate_model(lgb_model, X_train, y_train, dataset_name="Train")

print("  > Evaluating on VALIDATION set...")
val_metrics_lgb = evaluate_model(lgb_model, X_val, y_val, dataset_name="Validation")

leaderboard.append({
    "Model": "LightGBM",
    "Train_F1": train_metrics_lgb["F1_Weighted"],
    "Val_F1": val_metrics_lgb["F1_Weighted"],
    "Val_LogLoss": val_metrics_lgb["Log_Loss"]
})



=== Model 3: XGBoost Classifier ===
  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score (Weighted): 1.0000
    ROC-AUC (OvR):       1.0000
    Log Loss:            0.0021
  > Evaluating on VALIDATION set...
[Validation Performance]
    F1-Score (Weighted): 0.9964
    ROC-AUC (OvR):       1.0000
    Log Loss:            0.0092

=== Model 4: LightGBM Classifier ===
  > Evaluating on TRAINING set...
[Train Performance]
    F1-Score (Weighted): 0.9999
    ROC-AUC (OvR):       1.0000
    Log Loss:            0.0044
  > Evaluating on VALIDATION set...
[Validation Performance]
    F1-Score (Weighted): 0.9967
    ROC-AUC (OvR):       1.0000
    Log Loss:            0.0097

=== 🏆 Phase A Leaderboard: Default Models ===
| Model               |   Train_F1 |   Val_F1 |   Val_LogLoss |
|:--------------------|-----------:|---------:|--------------:|
| LightGBM            |   0.999908 | 0.996715 |    0.00968221 |
| XGBoost             |   1        | 0.996358 |    0.00920968 |
| Random

In [7]:
# --- SAVE PHASE A RESULTS ---
leaderboard_path = f"{DATA_DIR}phase_a_leaderboard.csv"
df_leaderboard = pd.DataFrame(leaderboard)

# Save to CSV
df_leaderboard.to_csv(leaderboard_path, index=False)
print(f"\n[SAVED] Phase A Leaderboard saved to: {leaderboard_path}")

# Display the table (Sorted by Val_F1)
print(df_leaderboard.sort_values(by="Val_F1", ascending=False).to_markdown(index=False))


[SAVED] Phase A Leaderboard saved to: data/phase_a_leaderboard.csv
| Model               |   Train_F1 |   Val_F1 |   Val_LogLoss |
|:--------------------|-----------:|---------:|--------------:|
| LightGBM            |   0.999908 | 0.996715 |    0.00968221 |
| XGBoost             |   1        | 0.996358 |    0.00920968 |
| Random Forest       |   1        | 0.995073 |    0.0229231  |
| Logistic Regression |   0.950453 | 0.950567 |    0.118396   |


## **I.5. Conclusion: Baseline Model Selection**

We successfully evaluated four distinct algorithms on the stratified datasets. The goal was to select a single "Champion" model for hyperparameter tuning based on **Generalization** (Validation F1) and **Confidence** (Log Loss).

**The Leaderboard:**

| Rank | Model | Val F1-Score | Val Log-Loss | Interpretation |
| :--- | :--- | :--- | :--- | :--- |
| **1** | **LightGBM** | **0.9967** | 0.0097 | **The Winner.** Highest accuracy and extremely fast convergence. |
| **2** | **XGBoost** | 0.9964 | **0.0092** | Very close second. Slightly better calibrated probabilities (lower Log Loss) but marginally lower F1. |
| **3** | **Random Forest** | 0.9951 | 0.0229 | Excellent accuracy, but significantly worse Log Loss (0.02 vs 0.009), indicating it is "less sure" of its predictions than boosting models. |
| **4** | **Logistic Regression** | 0.9506 | 0.1184 | The linear baseline performed surprisingly well (95%), proving the clusters have strong linear separability, but it cannot capture the edge cases like the tree models. |

**Key Findings:**
1.  **The "Surrogate" Validation:** All tree-based models achieved >99.5% accuracy. This effectively proves that **Group 1's clusters are deterministic.** The clusters are not random noise; they follow a strict logic based on the input features, which the models successfully reverse-engineered.
2.  **Boosting > Bagging:** Both Gradient Boosting methods (LightGBM, XGBoost) outperformed Random Forest in **Log Loss** by a factor of 2. They are not just guessing the right class; they are assigning it with near-100% probability.
3.  **Selection Decision:** We select **XGBoost** for the Tuning Phase.
    *   *Why not LightGBM?* Although LightGBM won on F1 by a tiny margin (0.0003), XGBoost has the **lowest Log Loss (0.0092)** and creates the most robust **SHAP plots**, which is our ultimate goal. (XGBoost's `TreeExplainer` is the gold standard for interpretability).

$\Rightarrow$ **Next Step:** We proceed to **Phase B: Hyperparameter Tuning of XGBoost** to prevent overfitting and ensure the model remains stable when we strip away features later.

# **II. Hyperparameter Tuning (XGBoost)**

## **II.1. Tuning Strategy: Bayesian Optimization**

Instead of a brute-force Grid Search (which is slow and inefficient), we employ **Tree-structured Parzen Estimator (TPE)** via the **Optuna** framework.

*   **The Objective:** Minimize **Log Loss** on the Validation Set.
    *   *Why Log Loss?* F1-Score is a "step function" (hard classes), making it difficult for an optimizer to find gradients. Log Loss is continuous and penalizes the model for being "uncertain," providing a smoother path to the global optimum.
*   **The Hyperparameter Search Space:**
    *   **Structure:** `max_depth` (Tree complexity), `min_child_weight` (Leaf node minimum mass).
    *   **Regularization:** `gamma` (Split threshold), `reg_alpha` (L1), `reg_lambda` (L2).
    *   **Sampling:** `subsample` (Rows), `colsample_bytree` (Features).
    *   **Learning:** `learning_rate` (Step size), `n_estimators` (Number of trees).

## **II.2. Code Implementation: Optuna Study & Final Model Training**

**Purpose:** Run 20 trials to find the optimal configuration that minimizes Log Loss without overfitting. Train the definitive `best_xgb_model` using the parameters found above and perform a final check on the Validation set.

In [6]:
import optuna
import xgboost as xgb
import json
import os
# from sklearn.metrics import log_loss

# --- CONFIGURATION ---
PARAMS_FILE = "data/best_xgb_params.json"

def objective(trial):
    params = {
        'objective': 'multi:softprob',
        'num_class': 4,
        'n_jobs': -1,
        'random_state': 42,
        'verbosity': 0,
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
    }
    
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_val)
    return log_loss(y_val, y_prob)

# --- EXECUTION LOGIC ---
if os.path.exists(PARAMS_FILE):
    print(f"--- [INFO] Found saved parameters in {PARAMS_FILE}. Skipping optimization. ---")
    with open(PARAMS_FILE, 'r') as f:
        best_params = json.load(f)
else:
    print("--- [START] Hyperparameter Tuning with Optuna ---")
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=20)
    
    best_params = study.best_params
    
    # Save to JSON
    with open(PARAMS_FILE, 'w') as f:
        json.dump(best_params, f, indent=4)
    print(f"--- [SAVED] Best parameters saved to {PARAMS_FILE} ---")

print("\n--- [RESULT] Best Parameters Loaded ---")
for key, value in best_params.items():
    print(f"    {key}: {value}")

# --- TRAIN FINAL MODEL ---
print("\n=== Training Final Champion Model ===")
# Ensure static params are added back (as they aren't optimized)
best_params.update({
    'objective': 'multi:softprob',
    'num_class': 4,
    'n_jobs': -1,
    'random_state': 42
})

final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

# --- FINAL VALIDATION CHECK ---
print("  > Evaluating Best Model on VALIDATION set...")
final_metrics = evaluate_model(final_model, X_val, y_val, dataset_name="Final Validation")

# Calculate Lift
default_logloss = 0.0092 # From Phase A
lift = ((default_logloss - final_metrics['Log_Loss']) / default_logloss) * 100

print(f"\n[Conclusion] Tuning Improvement:")
print(f"    Default Log Loss: {default_logloss}")
print(f"    Tuned Log Loss:   {final_metrics['Log_Loss']:.5f}")
print(f"    Performance Lift: {lift:.2f}%")

--- [START] Hyperparameter Tuning with Optuna ---
--- [SAVED] Best parameters saved to data/best_xgb_params.json ---

--- [RESULT] Best Parameters Loaded ---
    max_depth: 9
    min_child_weight: 7
    learning_rate: 0.12688996739620345
    n_estimators: 846
    gamma: 0.032537918839191105
    reg_alpha: 4.714958436497499
    reg_lambda: 3.237417655214124
    subsample: 0.8148533995968621
    colsample_bytree: 0.946760070343784

=== Training Final Champion Model ===
  > Evaluating Best Model on VALIDATION set...
[Final Validation Performance]
    F1-Score (Weighted): 0.9964
    ROC-AUC (OvR):       1.0000
    Log Loss:            0.0107

[Conclusion] Tuning Improvement:
    Default Log Loss: 0.0092
    Tuned Log Loss:   0.01073
    Performance Lift: -16.64%


The output is **correct**, but it reveals a critical nuance: **The Tuning actually made the model slightly "worse"** (Negative Lift: -16.64%).

**Why did this happen?**
*   **Default Log Loss (0.0092):** The default XGBoost is extremely aggressive (unconstrained depth, no regularization). It fits the data perfectly.
*   **Tuned Log Loss (0.0107):** Our Optuna search added **Regularization** (`reg_alpha`, `reg_lambda`, `gamma`).
*   **Interpretation:** The tuned model is "less confident" (higher log loss) but **more robust**. The default model likely "overfit" the probabilities (e.g., predicting 99.99% certainty), while the tuned model is more conservative (e.g., predicting 98% certainty).
*   **Is this bad?** No. In a production environment, we prefer the regularized model because it is less likely to break on unseen data. However, for a *Surrogate Model* whose only job is to mimic the clustering logic, the "Default" model might actually be superior because the clustering logic is static.


The previous tuning result failed to beat the default because the **Search Space was too conservative**. It forced the model to simplify itself, whereas the "Default" model was allowed to be complex.

**The Optimization Strategy: "Beat the Default"**

1.  **Enqueue the Default:** We will explicitly tell Optuna to try the Default Parameters *first*. This guarantees the result will be **at least as good** as the default. It mathematically cannot be worse.
2.  **Relax Constraints:** We will allow `max_depth` to go deeper (up to 15) and allow regularization to drop to effectively zero.
3.  **Increase Trials:** Bump to **50 trials** to explore the space more thoroughly.

In [8]:
# --- CONFIGURATION ---
PARAMS_FILE = "data/best_xgb_params_v2.json" # New file name to avoid conflict

def objective(trial):
    params = {
        'objective': 'multi:softprob',
        'num_class': 4,
        'n_jobs': -1,
        'random_state': 42,
        'verbosity': 0,
        
        # BROADER Search Space
        'max_depth': trial.suggest_int('max_depth', 4, 15), # Allow deeper trees
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1500),
        
        # LOWER Regularization Floor (Allowing it to be 0 like default)
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 2.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 2.0, log=True),
        
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
    }
    
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_val)
    return log_loss(y_val, y_prob)

# --- EXECUTION LOGIC ---
if os.path.exists(PARAMS_FILE):
    print(f"--- [INFO] Found saved parameters in {PARAMS_FILE}. Skipping optimization. ---")
    with open(PARAMS_FILE, 'r') as f:
        best_params = json.load(f)
else:
    print("--- [START] Hyperparameter Tuning (Aggressive) ---")
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='minimize')
    
    # 1. TRICK: Tell Optuna to try the "Default-ish" params first
    # This ensures we start with a strong baseline
    study.enqueue_trial({
        'max_depth': 6,
        'min_child_weight': 1,
        'learning_rate': 0.3,
        'n_estimators': 100,
        'gamma': 1e-8,
        'reg_alpha': 1e-8,
        'reg_lambda': 1.0,
        'subsample': 1.0,
        'colsample_bytree': 1.0
    })
    
    # 2. Run Optimization
    study.optimize(objective, n_trials=30) # Increased to 30 for better coverage
    
    best_params = study.best_params
    
    # Save to JSON
    with open(PARAMS_FILE, 'w') as f:
        json.dump(best_params, f, indent=4)
    print(f"--- [SAVED] Best parameters saved to {PARAMS_FILE} ---")

print("\n--- [RESULT] Best Parameters Loaded ---")
for key, value in best_params.items():
    print(f"    {key}: {value}")

# --- TRAIN FINAL MODEL ---
print("\n=== Training Final Champion Model ===")
best_params.update({
    'objective': 'multi:softprob',
    'num_class': 4,
    'n_jobs': -1,
    'random_state': 42
})

final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

# --- FINAL VALIDATION CHECK ---
print("  > Evaluating Best Model on VALIDATION set...")
final_metrics = evaluate_model(final_model, X_val, y_val, dataset_name="Final Validation")

# Calculate Lift
default_logloss = 0.0092 # From Phase A
lift = ((default_logloss - final_metrics['Log_Loss']) / default_logloss) * 100

print(f"\n[Conclusion] Tuning Improvement:")
print(f"    Default Log Loss: {default_logloss}")
print(f"    Tuned Log Loss:   {final_metrics['Log_Loss']:.5f}")
print(f"    Performance Lift: {lift:.2f}%")

--- [START] Hyperparameter Tuning (Aggressive) ---
--- [SAVED] Best parameters saved to data/best_xgb_params_v2.json ---

--- [RESULT] Best Parameters Loaded ---
    max_depth: 12
    min_child_weight: 4
    learning_rate: 0.033034837496608356
    n_estimators: 1321
    gamma: 1.47721632417482e-07
    reg_alpha: 0.19108536698828876
    reg_lambda: 8.427536862316056e-05
    subsample: 0.8655603970394049
    colsample_bytree: 0.9675651912854055

=== Training Final Champion Model ===
  > Evaluating Best Model on VALIDATION set...
[Final Validation Performance]
    F1-Score (Weighted): 0.9971
    ROC-AUC (OvR):       1.0000
    Log Loss:            0.0086

[Conclusion] Tuning Improvement:
    Default Log Loss: 0.0092
    Tuned Log Loss:   0.00863
    Performance Lift: 6.17%


### **II.3. Conclusion: Hyperparameter Optimization**

We moved beyond the default settings to rigorously optimize the XGBoost architecture using **Bayesian Optimization (Optuna)**. By expanding the search space and reducing constraints, we allowed the model to find a more complex and precise configuration.

**Optimization Results:**
*   **The Winning Configuration:** The algorithm converged on a "Deep & Slow" learning strategy:
    *   **`max_depth: 12`**: Significantly deeper than the default (6), allowing the model to capture highly complex, non-linear interaction effects between features.
    *   **`learning_rate: 0.033`**: A low learning rate (vs default 0.3) combined with high **`n_estimators: 1321`** ensures precise convergence without overshooting the minima.
    *   **`gamma` & `reg_alpha` $\approx$ 0**: The optimizer determined that strict regularization was unnecessary; the signal in the data is strong and clean.

**Performance Impact:**

| Metric | Default Model | Tuned Model | Improvement |
| :--- | :--- | :--- | :--- |
| **Log Loss (Confidence)** | 0.00920 | **0.00863** | **+6.17% Lift** |
| **F1-Score (Accuracy)** | 0.9964 | **0.9971** | **+0.07% Lift** |

**Key Findings:**
*   **Precision Engineering:** The positive lift of **6.17%** in Log Loss proves that the default model was slightly "under-fitted" to the nuances of the clusters. The tuned model is statistically more confident in its predictions.
*   **Surrogate Validity:** With an F1-Score of **99.71%**, the model effectively becomes a "Digital Twin" of the clustering logic. We can now be certain that explaining this model is equivalent to explaining the clusters themselves.

$\Rightarrow$ **Verdict:** We deploy the **Tuned XGBoost (v2)** as the finalized engine for the Feature Importance & SHAP analysis.

# **III. Visualization (Presentation)**

In [22]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import os

# Create directory for saving figures
FIGURES_DIR = "figures/"
os.makedirs(FIGURES_DIR, exist_ok=True)

# --- PREPARE DATA ---
# Re-construct the data
leaderboard_data = {
    "Model": ["LightGBM", "XGBoost (Default)", "Random Forest", "Logistic Regression"],
    "Val_F1": [0.9967, 0.9964, 0.9951, 0.9506],
    "Val_LogLoss": [0.0097, 0.0092, 0.0229, 0.1184]
}
df_lead = pd.DataFrame(leaderboard_data)

new_row = {
    "Model": "XGBoost (Tuned v2)", 
    "Val_F1": 0.9971, 
    "Val_LogLoss": 0.0086
}
df_final_viz = pd.concat([df_lead, pd.DataFrame([new_row])], ignore_index=True)
df_final_viz = df_final_viz.sort_values("Val_LogLoss", ascending=True)

# --- FIGURE 4: IMPROVED BAR CHART (Better Gradient) ---
# Custom scale: Start at Dark Blue (#08306b) -> End at Light Blue (#bdd7e7)
# This prevents the lightest bar from becoming white/invisible.
custom_blue_scale = [
    (0.0, "#08306b"), # Lowest LogLoss (Best) -> Darkest Blue
    (1.0, "#bdd7e7")  # Highest LogLoss (Worst) -> Lightest Blue (Visible)
]

fig4 = px.bar(
    df_final_viz,
    x="Val_LogLoss",
    y="Model",
    orientation="h",
    title="<b>Model Confidence (Log Loss): Lower is Better</b><br><sup>The Tuned XGBoost achieves the highest certainty (Lowest Error).</sup>",
    text_auto=".4f",
    color="Val_LogLoss",
    color_continuous_scale=custom_blue_scale
)

fig4.update_layout(
    height=600,
    width=1400,
    xaxis_title="<b>Validation Log Loss (Lower = Better)</b>",
    yaxis_title="<b>Model</b>",
    showlegend=False,
    coloraxis_showscale=True,  # Show the colorbar for Val_LogLoss
    coloraxis_colorbar=dict(
        title=dict(text="Val LogLoss", side="right", font=dict(size=16)),
        tickfont=dict(size=14)
    ),
    margin=dict(r=100, l=230),
    title_font_size=22,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    xaxis_tickfont_size=16,
    yaxis_tickfont_size=16
)

# Save figure 4 at high quality
fig4.write_image(f"{FIGURES_DIR}figure4_model_leaderboard.png", scale=3, width=1400, height=600)
print(f"✓ Figure 4 saved to {FIGURES_DIR}figure4_model_leaderboard.png")

fig4.show()

# --- FIGURE 5: ACCURACY vs CONFIDENCE (Clean Sidebar Version) ---

# 1. Sort Data by Performance (Best to Worst) so the list is ordered
# We sort by Log Loss (Ascending) -> Lowest Loss is First
df_viz_sorted = df_final_viz.sort_values("Val_LogLoss", ascending=True).reset_index(drop=True)

# 2. Define Colors
model_colors = {
    "Logistic Regression": "#E7B142", # Yellow/Orange
    "Random Forest": "#9467bd",       # Purple
    "LightGBM": "#2ca02c",            # Green
    "XGBoost (Default)": "#1f77b4",   # Blue
    "XGBoost (Tuned v2)": "#d62728"   # Red (Champion)
}

fig5 = px.scatter(
    df_viz_sorted,
    x="Val_LogLoss",
    y="Val_F1",
    color="Model",
    size=[70, 50, 50, 45, 45], # Champion gets biggest dot
    opacity=0.75,
    color_discrete_map=model_colors,
    title="<b>The Trade-off: Accuracy vs. Confidence</b><br><sup>Top-Left is the 'Sweet Spot'. Tuning pushed XGBoost further into the ideal zone.</sup>",
    hover_data=["Model", "Val_F1", "Val_LogLoss"]
)

# 3. Create Custom Sidebar Annotations
annotations = []
# Start Y position at the top (1.0) and move down
y_pos = 1.0 
y_step = 0.12 # Space between items

for index, row in df_viz_sorted.iterrows():
    model_name = row["Model"]
    color = model_colors.get(model_name, "black")
    
    # Add Champion Icon for the winner
    if "Tuned" in model_name:
        display_text = f"<b>🏆 {model_name}</b>"
    else:
        display_text = f"{model_name}"

    # Use HTML to color the dot inside the text
    # Unicode Circle: ●
    annotation_text = f"<span style='color:{color}; font-size:20px;'>●</span> {display_text}"

    annotations.append(dict(
        x=1.02, # Just outside the right edge of the plot area
        y=y_pos,
        xref="paper",
        yref="paper",
        text=annotation_text,
        showarrow=False,
        xanchor="left",
        yanchor="top",
        align="left",
        font=dict(size=16, color="#2c3e50")
    ))
    
    y_pos -= y_step # Move down for next item

fig5.update_layout(
    height=600,
    width=1400, # Wider to fit the sidebar
    xaxis_title="<b>Log Loss (Uncertainty) → Lower is Better</b>",
    yaxis_title="<b>F1-Score (Accuracy) → Higher is Better</b>",
    xaxis=dict(autorange="reversed"), # Best (Low Loss) on Right (or Left depending on preference, usually Left is 0) -> Let's keep Low Loss on Left
    showlegend=False, # We built our own custom legend
    annotations=annotations,
    margin=dict(r=270, l=98), # Large Right Margin to hold the text list
    title_font_size=22,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    xaxis_tickfont_size=16,
    yaxis_tickfont_size=16
)

# Force X-axis range to breathe if dots are on edge
min_loss = df_viz_sorted['Val_LogLoss'].min()
max_loss = df_viz_sorted['Val_LogLoss'].max()
fig5.update_xaxes(range=[max_loss * 1.1, min_loss * 0.5]) # Reversed range manual control

# Save figure 5 at high quality
fig5.write_image(f"{FIGURES_DIR}figure5_tuning_lift.png", scale=3, width=1400, height=600)
print(f"✓ Figure 5 saved to {FIGURES_DIR}figure5_tuning_lift.png")

fig5.show()

print(f"\n{'='*60}")
print(f"All 2 figures saved successfully in '{FIGURES_DIR}' directory")
print(f"Ready for presentation!")
print(f"{'='*60}")

✓ Figure 4 saved to figures/figure4_model_leaderboard.png


✓ Figure 5 saved to figures/figure5_tuning_lift.png



All 2 figures saved successfully in 'figures/' directory
Ready for presentation!
